In [1]:
!git clone https://github.com/IMvision12/Image-Super-Resolution.git

Cloning into 'Image-Super-Resolution'...
remote: Enumerating objects: 423, done.
remote: Counting objects: 100% (383/383), done.
remote: Compressing objects: 100% (239/239), done.
remote: Total 423 (delta 179), reused 302 (delta 133), pack-reused 40
Receiving objects: 100% (423/423), 134.26 MiB | 19.70 MiB/s, done.
Resolving deltas: 100% (187/187), done.


In [2]:
cd Image-Super-Resolution

/kaggle/working/Image-Super-Resolution


In [3]:
import tensorflow as tf
from dataset import download_data
from models import Discriminator, SRResNet
from train import train_srgan
from tqdm import tqdm

In [4]:
train_ds, val_ds = download_data()

EXTRACTING {'train_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip', 'valid_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X4.zip', 'train_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip', 'valid_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip'}


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/800 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/div2k/bicubic_x4/2.0.0.incompleteF9GUKX/div2k-train.tfrecord*...:   0%|   …

Generating validation examples...:   0%|          | 0/100 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/div2k/bicubic_x4/2.0.0.incompleteF9GUKX/div2k-validation.tfrecord*...:   0…

Dataset div2k downloaded and prepared to /root/tensorflow_datasets/div2k/bicubic_x4/2.0.0. Subsequent calls will reuse this data.


In [5]:
generator = tf.keras.models.load_model("srresnet-gen/model.h5")
discriminator = Discriminator()
trainer = train_srgan(generator, discriminator)

80134624/80134624 [==============================] - 4s 0us/step


In [6]:
for epoch in range(200):

    for lr, hr in train_ds.take(1000):
        perceptual_loss, loss_disc, psnr_value = trainer.train_step(lr, hr)

    if epoch % 10 == 0 or epoch == 199:
        print(f'Epochs : {epoch}   ||   Generator Loss : {perceptual_loss}   ||   Discriminator Loss : {loss_disc:.5f}   ||   PSNR : {psnr_value:.5f}')

Epochs : 0   ||   Generator Loss : 0.11393405497074127   ||   Discriminator Loss : 0.53974   ||   PSNR : 14.77560
Epochs : 10   ||   Generator Loss : 0.07889808714389801   ||   Discriminator Loss : 0.02986   ||   PSNR : 18.30664
Epochs : 20   ||   Generator Loss : 0.15275350213050842   ||   Discriminator Loss : 0.29038   ||   PSNR : 23.57341
Epochs : 30   ||   Generator Loss : 0.1494440883398056   ||   Discriminator Loss : 0.01810   ||   PSNR : 20.01568
Epochs : 40   ||   Generator Loss : 0.10409484058618546   ||   Discriminator Loss : 0.13835   ||   PSNR : 33.52646
Epochs : 50   ||   Generator Loss : 0.08683373779058456   ||   Discriminator Loss : 0.02220   ||   PSNR : 29.36370
Epochs : 60   ||   Generator Loss : 0.09693526476621628   ||   Discriminator Loss : 0.02261   ||   PSNR : 30.16919
Epochs : 70   ||   Generator Loss : 0.05137569084763527   ||   Discriminator Loss : 0.10252   ||   PSNR : 16.25770
Epochs : 80   ||   Generator Loss : 0.059374526143074036   ||   Discriminator Loss

In [7]:
!mkdir saved_model
generator.save('saved_model/',save_format='tf')
generator.save("model.h5")